# Transportation

## Overview

In this problem, consider a set of $I$ suppliers and a set of $J$ customers. Each supplier $i$ has a limited capacity $b_{i}$ and each customer $j$ has a certain demand $d_{j}$. The transportation costs from $i$ to $j$ are given by $c_{i, j}$.

$$
\begin{align}
    \text{min} \quad & \sum_{i \in I} \sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \; i \in I \;\forall \; j \in J \\
\end{align}
$$

## Example

Consider the following example:

Table 1: Customer demands
| Customer | Demand (lb/day) |
| --- | --- |
| A | 5 |
| B | 15 |
| C | 13 |
| D | 17 |

 
Table 2: Supplier availablity
| Supplier | Availability ($/lb) |
| --- | --- |
| 1 | 14 |
| 2 | 26 |
| 3 | 11 |


Table 3: Costs
| Supplier \ Customer | A | B | C | D |
| --- | --- | --- | --- | --- |
| 1 | 10 | 5 | 20 | 12 |
| 2 | 12 | 7 | 12 | 19 |
| 3 | 6 | 12 | 16 | 17 |

In [1]:
import pandas as pd
import pyomo.environ as pyo

In [2]:
costs = pd.DataFrame({
    "A": [10, 12, 6],
    "B": [5, 7, 12],
    "C": [20, 12, 16],
    "D": [12, 19, 17],
}, index=[1, 2, 3])

In [3]:
model = pyo.ConcreteModel()

In [4]:
# Create sets
model.I = pyo.Set(initialize=[1, 2, 3])
model.J = pyo.Set(initialize=["A", "B", "C", "D"])

In [5]:
# Create parameters
availability = {1: 14, 2: 26, 3: 11}
model.b = pyo.Param(model.I, initialize=availability)

demands = {"A": 5, "B": 15, "C": 13, "D": 17}
model.d = pyo.Param(model.J, initialize=demands)

c = {(i, j): costs.loc[i, j] for i in costs.index for j in costs.columns}
model.c = pyo.Param(model.I, model.J, initialize=c)

In [6]:
# Decision variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)

In [7]:
# Constraints
def availability_rule(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.b[i]

model.availability_constr = pyo.Constraint(model.I, rule=availability_rule)

def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.I) == model.d[j]

model.demand_constr = pyo.Constraint(model.J, rule=demand_rule)

In [8]:
# Objective
model.obj = pyo.Objective(expr=sum(model.x[i, j] * model.c[i, j] for (i, j) in model.x), sense=pyo.minimize)

In [9]:
# Solve
cbc = pyo.SolverFactory("cbc")
sol = cbc.solve(model, tee=False)

In [10]:
results = pd.DataFrame(index=model.I, columns=model.J)
for i, j in model.x:
    results.loc[i, j] = model.x[i, j].value

In [11]:
results

,A,B,C,D
1,0.0,2.0,0.0,12.0
2,0.0,13.0,13.0,0.0
3,5.0,0.0,0.0,5.0
